In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/scamps_datatape_32x36x36_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_32x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_tscan(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_tscan(train).cache(f'{tmp}/tscan'), to_tf_tscan(valid).cache(f'{tmp}/tscan')
train, valid = to_tf_tscan(train), to_tf_tscan(valid)

In [17]:
ts_can_e2e = TS_CAN_end_to_end(n=1)
ts_can_e2e.compile(optimizer=keras.optimizers.SGD(learning_rate=0.001), loss='mse')
ts_can_e2e.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(ts_can_e2e, input_sig=[tf.TensorSpec([32, 36, 36, 3])])/32:.0f}')

Flops per frame:53434160


In [21]:
ts_can_e2e.fit(train.shuffle(2000), validation_data=valid, epochs=1, callbacks=[keras.callbacks.ModelCheckpoint('../weights/TS-CAN_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
ts_can_e2e.load_weights('../weights/TS-CAN_SCAMPS.h5')

37909/37909 [==============================] - 672s 18ms/step - loss: 0.1777 - val_loss: 0.1545

Epoch 00001: val_loss improved from inf to 0.15448, saving model to ../weights\TS-CAN_SCAMPS.h5


In [ ]:
ts_can_e2e.load_weights('../weights/TS-CAN_SCAMPS.h5')

In [10]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [22]:
eval_on_dataset(test_set_PURE, ts_can_e2e, 32, (36, 36), step=1, batch=0, save='../results/TS-CAN_SCAMPS_PURE.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/TS-CAN_SCAMPS_PURE.h5')
get_metrics('../results/TS-CAN_SCAMPS_PURE.h5')

100%|██████████| 59/59 [00:36<00:00,  1.64it/s]


{'Sliding window': {'MAE': 23.824, 'RMSE': 29.806, 'R': -0.00331},
 'Whole video': {'MAE': 22.746, 'RMSE': 28.844, 'R': -0.18294}}

In [23]:
eval_on_dataset(test_set_UBFC_rPPG2, ts_can_e2e, 32, (36, 36), step=1, batch=0, save='../results/TS-CAN_SCAMPS_UBFC.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/TS-CAN_SCAMPS_UBFC.h5')
get_metrics('../results/TS-CAN_SCAMPS_UBFC.h5')

100%|██████████| 42/42 [00:23<00:00,  1.80it/s]


{'Sliding window': {'MAE': 17.736, 'RMSE': 25.783, 'R': 0.24157},
 'Whole video': {'MAE': 16.666, 'RMSE': 24.613, 'R': 0.27253}}